In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../..')

In [3]:
import asyncio
import logging

from app.redis_transcribe.connection import get_redis_client
from app.settings import settings
from app.services.transcription.processor import Processor




redis_client = await get_redis_client(settings.redis_host, settings.redis_port,settings.redis_password)

processor = Processor(redis_client,max_length=30)



2025-02-26 16:23:01,962 - INFO - Attempting Redis connection to redis:6379
2025-02-26 16:23:01,963 - INFO - Redis password is configured
2025-02-26 16:23:01,967 - INFO - Successfully connected to Redis server


In [4]:
from app.services.audio.redis_models import TranscriptStore

In [5]:
import pandas as pd
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000
pd.options.display.max_colwidth = 1000

In [6]:
from app.services.audio.redis_models import Connection, Meeting, Transcriber

In [27]:
await redis_client.flushdb()

True

In [ ]:

transcripts = await TranscriptStore.get_raw_transcript_data(redis_client)
transcripts

NameError: name 'Transcript' is not defined

In [20]:
transcripts = await TranscriptStore.get_all_transcripts(redis_client)

2025-02-26 13:11:47,279 - WARNING - Invalid transcript data format for key Transcript:the-zdjv-byg: 1 validation error for TranscriptSegmentModel
words
  Input should be a valid list [type=list_type, input_value=9.13, input_type=float]
    For further information visit https://errors.pydantic.dev/2.10/v/list_type
2025-02-26 13:11:47,279 - WARNING - Invalid transcript data format for key Transcript:the-zdjv-byg: 1 validation error for TranscriptSegmentModel
words
  Input should be a valid list [type=list_type, input_value=9.13, input_type=float]
    For further information visit https://errors.pydantic.dev/2.10/v/list_type
2025-02-26 13:11:47,280 - WARNING - Invalid transcript data format for key Transcript:the-zdjv-byg: 1 validation error for TranscriptSegmentModel
words
  Input should be a valid list [type=list_type, input_value=11.45, input_type=float]
    For further information visit https://errors.pydantic.dev/2.10/v/list_type
2025-02-26 13:11:47,281 - WARNING - Invalid transcript

In [21]:
transcripts

[]

In [18]:
df = await TranscriptStore.get_all_transcripts_df(redis_client)
print(len(df))
df.drop(columns=['words']).tail(10)

2025-02-26 13:10:43,619 - WARNING - Invalid transcript data format for key Transcript:the-zdjv-byg: 1 validation error for TranscriptSegmentModel
words
  Input should be a valid list [type=list_type, input_value=9.13, input_type=float]
    For further information visit https://errors.pydantic.dev/2.10/v/list_type
2025-02-26 13:10:43,619 - WARNING - Invalid transcript data format for key Transcript:the-zdjv-byg: 1 validation error for TranscriptSegmentModel
words
  Input should be a valid list [type=list_type, input_value=9.13, input_type=float]
    For further information visit https://errors.pydantic.dev/2.10/v/list_type
2025-02-26 13:10:43,620 - WARNING - Invalid transcript data format for key Transcript:the-zdjv-byg: 1 validation error for TranscriptSegmentModel
words
  Input should be a valid list [type=list_type, input_value=11.45, input_type=float]
    For further information visit https://errors.pydantic.dev/2.10/v/list_type
2025-02-26 13:10:43,620 - WARNING - Invalid transcript

0


KeyError: "['words'] not found in axis"

In [8]:
df

""
